In [ ]:
#nothing here

In [24]:
import pandas as pd
import requests

In [115]:
# this codes gets the trains that go through alvsjo

url = "https://api.trafikinfo.trafikverket.se/v2/data.json"

query = """
<REQUEST>
  <LOGIN authenticationkey='d13943e691d644ba96f4dab8c2e42c09' />
  <QUERY objecttype="TrainAnnouncement" schemaversion="1.8" orderby="AdvertisedTimeAtLocation">
    <FILTER>
      <!-- Any train that goes via Älvsjö -->
      <EQ name="LocationSignature" value="S" />
      <!-- Time window -->
      <GT name="AdvertisedTimeAtLocation" value="2025-09-12T00:00:00.000+02:00" />
      <LT name="AdvertisedTimeAtLocation" value="2025-09-12T23:59:59.000+02:00" />
    </FILTER>
    <INCLUDE>AdvertisedTrainIdent</INCLUDE>
    <INCLUDE>AdvertisedTimeAtLocation</INCLUDE>
    <INCLUDE>EstimatedTimeAtLocation</INCLUDE>
    <INCLUDE>TimeAtLocation</INCLUDE>
    <INCLUDE>LocationSignature</INCLUDE>
    <INCLUDE>TrackAtLocation</INCLUDE>
    <INCLUDE>Advertised</INCLUDE>
    <INCLUDE>Operator</INCLUDE>
    <INCLUDE>ActivityType</INCLUDE>
  </QUERY>
</REQUEST>
"""
response = requests.post(url, data=query.encode("utf-8"), headers={"Content-Type": "text/xml"})
data = response.json()

In [116]:
as_trains=pd.DataFrame(data['RESPONSE']['RESULT'][0]['TrainAnnouncement'])

In [117]:
as_trains

""


In [101]:
#this code makes a list of the trains that go through central and alvsjo, no pendeltags therefore since they stop at stockholm city

import requests
import pandas as pd
from datetime import datetime

# ==== SETTINGS ====
API_KEY = "d13943e691d644ba96f4dab8c2e42c09"
DATE = "2025-09-12"   # yyyy-mm-dd, change as needed
STATIONS = ["Cst", "Äs"]  # Stockholm C, Älvsjö
SCHEMAVERSION = "1.8"

# ==== FUNCTION TO QUERY TRAFIKVERKET ====
def get_train_events(station, date):
    url = "https://api.trafikinfo.trafikverket.se/v2/data.json"
    query = f"""
    <REQUEST>
      <LOGIN authenticationkey="{API_KEY}" />
      <QUERY objecttype="TrainAnnouncement" schemaversion="{SCHEMAVERSION}" orderby="AdvertisedTimeAtLocation">
        <FILTER>
          <EQ name="LocationSignature" value="{station}" />
          <GT name="AdvertisedTimeAtLocation" value="{date}T00:00:00.000+02:00" />
          <LT name="AdvertisedTimeAtLocation" value="{date}T23:59:59.000+02:00" />
        </FILTER>
        <INCLUDE>AdvertisedTrainIdent</INCLUDE>
        <INCLUDE>LocationSignature</INCLUDE>
        <INCLUDE>AdvertisedTimeAtLocation</INCLUDE>
        <INCLUDE>EstimatedTimeAtLocation</INCLUDE>
        <INCLUDE>TimeAtLocation</INCLUDE>
        <INCLUDE>Advertised</INCLUDE>
        <INCLUDE>Operator</INCLUDE>
        <INCLUDE>TrackAtLocation</INCLUDE>
      </QUERY>
    </REQUEST>
    """
    resp = requests.post(url, data=query.encode("utf-8"), headers={"Content-Type": "text/xml"})
    data = resp.json()

    # Flatten into a DataFrame
    announcements = data["RESPONSE"]["RESULT"][0]["TrainAnnouncement"]
    df = pd.DataFrame(announcements)
    df["station"] = station
    return df




In [62]:
# ==== GET DATA FOR BOTH STATIONS ====
dfs = [get_train_events(st, DATE) for st in STATIONS]
df_all = pd.concat(dfs, ignore_index=True)

# ==== CLEAN & CONVERT TIME FIELDS ====
for col in ["AdvertisedTimeAtLocation", "EstimatedTimeAtLocation", "TimeAtLocation","Operator"]:
    df_all[col] = pd.to_datetime(df_all[col], errors="coerce")

# ==== PIVOT: ONE ROW PER TRAIN, BOTH STATIONS ====
timetable = df_all.pivot_table(
    index="AdvertisedTrainIdent",
    columns="station",
    values=["AdvertisedTimeAtLocation", "EstimatedTimeAtLocation", "TimeAtLocation", "Advertised","Operator"],
    aggfunc="first"
)

timetable.reset_index(inplace=True)

# ==== RESULT ====
print(timetable)

        AdvertisedTrainIdent Advertised         AdvertisedTimeAtLocation  \
station                             Cst     Äs                       Cst   
0                          1       True  False 2025-09-12 23:17:00+02:00   
1                         10       True    NaN 2025-09-12 12:12:00+02:00   
2                        100       True    NaN 2025-09-12 07:40:00+02:00   
3                        101       True    NaN 2025-09-12 16:07:00+02:00   
4                        102       True    NaN 2025-09-12 08:40:00+02:00   
...                      ...        ...    ...                       ...   
1060                     982       True    NaN 2025-09-12 22:05:00+02:00   
1061                     983       True    NaN 2025-09-12 21:47:00+02:00   
1062                     988       True    NaN 2025-09-12 23:05:00+02:00   
1063                     989       True    NaN 2025-09-12 22:49:00+02:00   
1064                     995       True    NaN 2025-09-12 23:47:00+02:00   

           

/tmp/ipython-input-1188336088.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_all[col] = pd.to_datetime(df_all[col], errors="coerce")


In [63]:
display(timetable)

AdvertisedTrainIdent Advertised         AdvertisedTimeAtLocation  \
station                             Cst     Äs                       Cst   
0                          1       True  False 2025-09-12 23:17:00+02:00   
1                         10       True    NaN 2025-09-12 12:12:00+02:00   
2                        100       True    NaN 2025-09-12 07:40:00+02:00   
3                        101       True    NaN 2025-09-12 16:07:00+02:00   
4                        102       True    NaN 2025-09-12 08:40:00+02:00   
...                      ...        ...    ...                       ...   
1060                     982       True    NaN 2025-09-12 22:05:00+02:00   
1061                     983       True    NaN 2025-09-12 21:47:00+02:00   
1062                     988       True    NaN 2025-09-12 23:05:00+02:00   
1063                     989       True    NaN 2025-09-12 22:49:00+02:00   
1064                     995       True    NaN 2025-09-12 23:47:00+02:00   

                                    EstimatedTimeAtLocation      \
station                        Äs                       Cst  Äs   
0       2025-09-12 23:23:00+02:00                       NaT NaT   
1                             NaT                       NaT NaT   
2                             NaT                       NaT NaT   
3                             NaT                       NaT NaT   
4                             NaT 2025-09-12 08:42:00+02:00 NaT   
...                           ...                       ...  ..   
1060                          NaT 2025-09-12 22:08:00+02:00 NaT   
1061                          NaT                       NaT NaT   
1062                          NaT                       NaT NaT   
1063                          NaT 2025-09-12 22:51:00+02:00 NaT   
1064                          NaT                       NaT NaT   

                   TimeAtLocation                            
station                       Cst                        Äs  
0       2025-09-12 23:17:00+02:00 2025-09-12 23:28:00+02:00  
1       2025-09-12 12:12:00+02:00                       NaT  
2       2025-09-12 07:39:00+02:00                       NaT  
3       2025-09-12 16:06:00+02:00                       NaT  
4       2025-09-12 08:43:00+02:00                       NaT  
...                           ...                       ...  
1060    2025-09-12 22:09:00+02:00                       NaT  
1061    2025-09-12 21:45:00+02:00                       NaT  
1062    2025-09-12 23:05:00+02:00                       NaT  
1063    2025-09-12 22:51:00+02:00                       NaT  
1064    2025-09-12 23:46:00+02:00                       NaT  

[1065 rows x 9 columns]